# Context managers exercises

# 1. Timer context manager

Create a `Timer` context manager that measures the running time of the `with` block. The context manager takes an optional name argument and prints the block's name at the end too. 

In [1]:
from datetime import datetime


class Timer(object):
    def __init__(self, name=None):
        self.name = name
        
    def __enter__(self):
        self.start = datetime.now()
        
    def __exit__(self, *args):
        running_time = datetime.now() - self.start
        if self.name is not None:
            print("{} ran for {} seconds".format(self.name, running_time.total_seconds()))
        else:
            print("unnamed ran for {} seconds".format(running_time.total_seconds()))
        
# prints "slow code ran for F seconds
# F is the total_seconds the block took to finish (float)
with Timer("slow code"):
    s = sum(range(100000))
    
# prints "unnamed ran for F seconds
with Timer():
    s = sum(range(100000))

slow code ran for 0.003206 seconds
unnamed ran for 0.002945 seconds


# Using context managers

Many built-in libraries provide context managers for unmanaged resources. We will try a few of them.

## 2. `gzip`

[Documentation](https://docs.python.org/3/library/gzip.html)

## 2.1

In Lecture 03, we downloaded a Wikipedia page and computed some statistics on it. Download the same page and save it to a gzip file using the gzip's module context manager.

In [2]:
import wikipedia
import gzip

article = wikipedia.page("Budapest")

with gzip.open("budapest.gz", "wt") as f:
    f.write(article.content)

## 2.2

Open the file for reading and open another for writing and copy all words longer than `N` from the first file into the second.

In [3]:
N = 10

# this is not how you should tokenize any text!

with gzip.open("budapest.gz", "rt") as infile, gzip.open("budapest_filtered.gz", "wt") as outfile:
    for line in infile:
        words = line.split(" ")
        long_words = filter(lambda w: len(w) > N, words)
        outfile.write(" ".join(long_words) + "\n")

# 3. `tempfile`

[Documentation](https://docs.python.org/3/library/tempfile.html)

##  3.1

Open a temporary file for writing and write a few lines into the file. Can you open and read the file after the `with` block? If not, find a solution in the documentation.

In [4]:
import tempfile

with tempfile.TemporaryFile("wt") as f:
    print(f.name)
    f.write("Hello world\n")
    
with tempfile.NamedTemporaryFile("wt", delete=False) as f:
    print(f.name)
    f.write("Hello world\n")
    
with open(f.name) as inf:
    print("Reading from tempfile: {}".format(f.name))
    print(inf.read())

56
/tmp/tmpfayu84xz
Reading from tempfile: /tmp/tmpfayu84xz
Hello world



## 3.2

Create a temporary directory. Download a few articles from Wikipedia and save them to separate files in the directory. Use the `os` modules (`os.path` submodule) for path handling.

In [5]:
import os

with tempfile.TemporaryDirectory() as d:
    for title in ["Budapest", "Vienna", "Berlin"]:
        aritcle = wikipedia.page(title)
        with open(os.path.join(d, title.lower()), "w") as f:
            f.write(article.content)
    print(os.listdir(d))

['berlin', 'vienna', 'budapest']


## 4. `DirectoryHandler`

Create a `DirectoryHandler` class that:

- takes a directory name as its constructor parameter. If the directory does not exist, it creates a new directory.
- has an `add_file` function that takes a filename and creates the file in the directory that the class handles. The function should return an open file handler for writing. If a file has already been opened by `add_file`, do not open a new file, return the existing handler.
- make it a context manager that makes sure that each file added via `add_file` is closed when the `with` block exits

In [6]:
import os

class DirectoryHandler:
    def __init__(self, dirname):
        self.dirname = dirname
        if not os.path.exists(dirname):
            os.makedirs(dirname)
        self.file_handlers = {}
        
    def __enter__(self):
        return self
    
    def __exit__(self, *args):
        for fh in self.file_handlers.values():
            fh.close()
            
    def add_file(self, filename):
        full_path = os.path.join(self.dirname, filename)
        # idiomatic solution
        return self.file_handlers.setdefault(filename, open(full_path, 'w'))
        # solution for normal people
        # if filename not in self.file_handlers:
        #     self.file_handlers[filename] = open(full_path, 'w')
        # return self.file_handlers[filename]
    
                
dirname = "dummy_dir"
with DirectoryHandler(dirname) as dhandler:
    fd = dhandler.add_file("f1")
    fd.write("abc\n")
    fd = dhandler.add_file("f1")
    fd.write("abc\n")
    fd = dhandler.add_file("f2")
    fd.write("def\n")
    
f1_name = os.path.join(dirname, "f1")

with open(f1_name) as f:
    assert f.read() == "abc\nabc\n"
    
f2_name = os.path.join(dirname, "f2")
with open(f2_name) as f:
    assert f.read() == "def\n"

In [7]:
# checking if files are closed

handlers = []

for i in range(10000):
    d = DirectoryHandler(dirname)
    handlers.append(d)
    with d as dhandler:
        dhandler.add_file("f3")